In [ ]:
import subprocess
import os

#Parameters for models
#Change them round here
#[ -round <T> ]	The number of rounds to train (default=300)
#[ -tc <k> ]	Number of threshold candidates to search. -1 to use all feature values (default=10)
#pick some small parameters first to make sure it is working

parameters={'-round':[ 100, 200,  300, 400, 500, 1000, 1500], '-tc': [5, 10, 15, 30, 50]}

#train model(s)
for round_ in parameters['-round']:
    for tc in parameters['-tc']:
        #train code
        print('train_code', 'round', round_,'tc',  tc)
        train_code = 'java -jar RankLib.jar -train MSLR-WEB10K/Fold1/train.txt -validate MSLR-WEB10K/Fold1/vali.txt -ranker 2 -norm zscore  -save RB_models/RB_model_r%s_tc%s.txt -round %s -tc %s' %(round_, tc, round_, tc)          
        subprocess.run(train_code.split())

def get_list_models(directory_name):
    list_models=[]
    for file in os.listdir(directory_name):
        if file.endswith(".txt"):
            list_models.append(os.path.join(directory_name, file))
    return list_models

models=get_list_models('RB_models/')

for model in models:
    test_code='java -jar RankLib.jar -load %s -rank MSLR-WEB10K/Fold1/test.txt -score test_RB_rank_score_%s'%(model, model[10:])
    output = subprocess.check_output(test_code.split())

ERR={}
for model in models:
    test_code='java -jar RankLib.jar -load %s -test MSLR-WEB10K/Fold1/test.txt -metric2T ERR@10 '%(model)
    output = subprocess.check_output(test_code.split())
    string_ouput=str(output)
    ERR['%s' %(model[10:])]=eval(string_ouput[-9:-3])
    
NDCG={}
for model in models:
    test_code='java -jar RankLib.jar -load %s -test MSLR-WEB10K/Fold1/test.txt -metric2T ERR@10 '%(model)
    output = subprocess.check_output(test_code.split())
    string_ouput=str(output)
    NDCG['%s' %(model[10:])]=eval(string_ouput[-9:-3])
    

#dictionary containing the model name and the NDCG value
ERR

ERR
#RB_model_r100_tc10.txt NDCG 0.3581
#RB_model_r100_tc10.txt ERR 0.2518
#RN_model_e20_l2_n1_lr5e-05.txt NDCG 0.215
#RN_model_e20_l2_n1_lr5e-05.txt ERR 0.1491
#LambdaMART1.txt NDCG 0.4452
#LambdaMART1.txt ERR 0.3449

In [ ]:

#Parameters for models
#Default paramers:
#[ -tree <t> ]	Number of trees (default=1000)
#[ -leaf <l> ]	Number of leaves for each tree (default=10)
#[ -shrinkage <factor> ]	Shrinkage, or learning rate (default=0.1)
#[ -tc <k> ]	Number of threshold candidates for tree spliting. -1 to use all feature values (default=256)
#[ -mls <n> ]	Min leaf support -- minimum #samples each leaf has to contain (default=1)
#[ -estop <e> ]	Stop early when no improvement is observed on validaton data in e consecutive rounds (default=100)

#to test it is working put in some small values for all of these before running anything too big
parameters={'-tree':[500, 850, 1000, 1250, 1500], '-leaf': [1,5, 10], '-shrinkage':[0.01], '-tc':[ 256], '-estop':[1]}

#Saves trained model(s)
for tree in parameters['-tree']:
    for leaf in parameters['-leaf']:
        for tc in parameters['-tc']:
            for estop in parameters['-estop']:
                #train code
                if os.path.isfile('LM_models/LM_model_t%s_l%s_tc%s_stop%s.txt'%(tree, leaf, tc, estop)):
                    print('LM_models/LM_model_t%s_l%s_tc%s_stop%s.txt'%(tree, leaf, tc, estop))
                    print('already run')
                else:
                    print('running model','tree', tree,'leaf',  leaf, 'tc', tc,'estop', estop)
                    train_code = 'java -jar RankLib.jar -train MSLR-WEB10K/Fold1/train.txt -validate MSLR-WEB10K/Fold1/vali.txt -ranker 6 -norm zscore -metric2t NDCG@10  -save LM_models/LM_model_t%s_l%s_tc%s_stop%s.txt -tree %s -leaf %s -tc %s -estop %s' %(tree, leaf, tc, estop, tree, leaf, tc, estop)          
                    subprocess.run(train_code.split())

models=get_list_models('LM_models/')

for model in models:
    test_code='java -jar RankLib.jar -load %s -rank MSLR-WEB10K/Fold1/test.txt -score test_RB_rank_score_%s'%(model, model[10:])
    output = subprocess.check_output(test_code.split())

ERR={}
for model in models:
    test_code='java -jar RankLib.jar -load %s -test MSLR-WEB10K/Fold1/test.txt -metric2T ERR@10 '%(model)
    output = subprocess.check_output(test_code.split())
    string_ouput=str(output)
    ERR['%s' %(model[10:])]=eval(string_ouput[-9:-3])
    
NDCG={}
for model in models:
    test_code='java -jar RankLib.jar -load %s -test MSLR-WEB10K/Fold1/test.txt -metric2T ERR@10 '%(model)
    output = subprocess.check_output(test_code.split())
    string_ouput=str(output)
    NDCG['%s' %(model[10:])]=eval(string_ouput[-9:-3])
    
#tests model(s) against NDCG


In [ ]:
#Parameters for models

#[ -epoch <T> ]	The number of epochs to train (default=100)
#[ -layer <layer> ]	The number of hidden layers (default=1)
#[ -node <node> ]	The number of hidden nodes per layer (default=10)
#[ -lr <rate> ]	Learning rate (default=0.00005)

#to test it is working put in some small values for all of these before running anything too big
parameters={'-epoch':[ 50, 100, 150, 200], '-layer':[1, 2,3,4,5], '-node':[5,10,15,20, 30], '-lr':[0.005, 0.00005]}

#trains model and saves it with the model name containing the parameter values
import pickle
for epoch in parameters['-epoch']:
    for layer in parameters['-layer']:
        for node in parameters['-node']:
            for learning_rate in parameters['-lr']:
                #train code
                if os.path.isfile('RN_models/RN_model_test_e%s_l%s_n%s_lr%s.txt'%(epoch, layer, node, learning_rate)):
                    print('RN_models/RN_model_test_e%s_l%s_n%s_lr%s.txt'%(epoch, layer, node, learning_rate))
                    print('already run')
                else:
                    print('train_code', 'epoch', epoch, 'layer', layer, 'node', node, 'learning_rate', learning_rate)
                    train_code = 'java -jar RankLib.jar -train MSLR-WEB10K/Fold1/train.txt -test MSLR-WEB10K/Fold1/test.txt -validate MSLR-WEB10K/Fold1/vali.txt -ranker 1 -norm zscore  -save RN_models/RN_model_test_e%s_l%s_n%s_lr%s.txt -epoch %s -layer %s -node %s -lr %s' %(epoch, layer, node, learning_rate, epoch, layer, node, learning_rate)          
                    subprocess.run(train_code.split())
                



#test code against NDCG criteris
models=get_list_models('RN_models/')

for model in models:
    test_code='java -jar RankLib.jar -load %s -rank MSLR-WEB10K/Fold1/test.txt -score test_RB_rank_score_%s'%(model, model[10:])
    output = subprocess.check_output(test_code.split())

ERR={}
for model in models:
    test_code='java -jar RankLib.jar -load %s -test MSLR-WEB10K/Fold1/test.txt -metric2T ERR@10 '%(model)
    output = subprocess.check_output(test_code.split())
    string_ouput=str(output)
    ERR['%s' %(model[10:])]=eval(string_ouput[-9:-3])
    
NDCG={}
for model in models:
    test_code='java -jar RankLib.jar -load %s -test MSLR-WEB10K/Fold1/test.txt -metric2T ERR@10 '%(model)
    output = subprocess.check_output(test_code.split())
    string_ouput=str(output)
    NDCG['%s' %(model[10:])]=eval(string_ouput[-9:-3])
    